In [4]:
import pandas as pd

In [6]:
# Read in Csvs

df = pd.read_csv('Statcast_2020.csv')
df19 = pd.read_csv('Statcast_2019.csv')

# Merge years and picks columns

df_merge = df.append(df19)
df_clean = df_merge[['pitch_type','release_speed','release_pos_x','release_pos_z','player_name','zone','game_type','stand','p_throws','type','balls','strikes','pfx_x','pfx_z','plate_x','plate_z','inning','vx0','vy0','ax','ay','az','sz_top','sz_bot','effective_speed','release_spin_rate','release_extension','release_pos_y','at_bat_number','pitch_number','pitch_name','spin_axis','delta_home_win_exp','delta_run_exp']]
df_clean.head()

,pitch_type,release_speed,release_pos_x,release_pos_z,player_name,zone,game_type,stand,p_throws,type,...,effective_speed,release_spin_rate,release_extension,release_pos_y,at_bat_number,pitch_number,pitch_name,spin_axis,delta_home_win_exp,delta_run_exp
0,FF,98.8,-0.41,6.46,"Fairbanks, Pete",14.0,F,L,R,X,...,100.2,2483.0,7.2,53.26,64,2,4-Seam Fastball,198.0,0.034,-0.370
1,FF,98.7,-0.33,6.62,"Fairbanks, Pete",12.0,F,L,R,B,...,100.0,2522.0,7.2,53.32,64,1,4-Seam Fastball,189.0,0.003,0.040
2,SL,89.6,-0.10,6.79,"Fairbanks, Pete",5.0,F,R,R,S,...,90.8,2537.0,6.9,53.56,63,6,Slider,58.0,0.053,-0.392
3,FF,100.4,-0.38,6.50,"Fairbanks, Pete",6.0,F,R,R,S,...,101.7,2469.0,7.1,53.37,63,5,4-Seam Fastball,191.0,0.000,-0.102
4,FF,97.6,-0.18,6.63,"Fairbanks, Pete",12.0,F,R,R,S,...,98.9,2339.0,7.1,53.42,63,4,4-Seam Fastball,191.0,0.000,-0.053


In [9]:
# Create Top Pitcher Databases

df_cole = df_clean[df_clean['player_name'] == 'Cole, Gerrit']
df_degrom = df_clean[df_clean['player_name'] == 'DeGrom, Jacob']
df_scherzer = df_clean[df_clean['player_name'] == 'Scherzer, Max']
df_bauer = df_clean[df_clean['player_name'] == 'Bauer, Trevor']
df_bieber = df_clean[df_clean['player_name'] == 'Bieber, Shane']
df_kershaw = df_clean[df_clean['player_name'] == 'Kershaw, Clayton']
df_lamet = df_clean[df_clean['player_name'] == 'Lamet, Dinelson']
df_nola = df_clean[df_clean['player_name'] == 'Nola, Aaron']
df_buehler = df_clean[df_clean['player_name'] == 'Buehler, Walker']


In [7]:
# Create L and R Handed Databases

df_r = df_clean[df_clean['p_throws'] == 'R']
df_l = df_clean[df_clean['p_throws'] == 'L']

In [10]:
# Save Top Pitcher Databases

df_cole.to_csv('resources/cole.csv')
df_degrom.to_csv('resources/degrom.csv')
df_scherzer.to_csv('resources/scherzer.csv')
df_bauer.to_csv('resources/bauer.csv')
df_bieber.to_csv('resources/bieber.csv')
df_kershaw.to_csv('resources/kershaw.csv')
df_lamet.to_csv('resources/lamet.csv')
df_nola.to_csv('resources/nola.csv')
df_buehler.to_csv('resources/buehler.csv')


In [8]:
# Save L and R Handed Pitcher Databases
df_l.to_csv('resources/l.csv')
df_r.to_csv('resources/r.csv')

In [43]:
#Create database sorted by team
df_teams = df_merge[['inning_topbot','home_team','away_team','pitch_type','release_speed','release_pos_x','release_pos_z','player_name','zone','game_type','stand','p_throws','type','balls','strikes','pfx_x','pfx_z','plate_x','plate_z','inning','vx0','vy0','ax','ay','az','sz_top','sz_bot','effective_speed','release_spin_rate','release_extension','release_pos_y','at_bat_number','pitch_number','pitch_name','spin_axis','delta_home_win_exp','delta_run_exp']]
df_top = df_teams[df_teams['inning_topbot']== 'Top']
df_bottom = df_teams[df_teams['inning_topbot']== 'Bottom']
df_hometeam = df_top.drop(columns = ['away_team','inning_topbot'])
df_awayteam = df_top.drop(columns = ['home_team','inning_topbot'])
df_hometeam['team'] = df_hometeam['home_team']
df_awayteam['team'] = df_awayteam['away_team']
df_teams = df_hometeam.append(df_awayteam)
df_teams = df_teams[df_teams['pitch_type'].isnull() == False]
df_teams = df_teams.drop(columns = ['home_team','away_team'])
df_teams = df_teams.sort_values('team')
df_teams

,pitch_type,release_speed,release_pos_x,release_pos_z,player_name,zone,game_type,stand,p_throws,type,...,release_spin_rate,release_extension,release_pos_y,at_bat_number,pitch_number,pitch_name,spin_axis,delta_home_win_exp,delta_run_exp,team
37703,FF,92.5,-1.44,5.56,"Javier, Cristian",11.0,R,L,R,S,...,2278.0,6.0,54.49,19,4,4-Seam Fastball,202.0,0.000,0.000,ARI
30524,CU,75.5,2.47,5.62,"Erlin, Robbie",12.0,R,R,L,S,...,2631.0,5.9,54.64,39,5,Curveball,332.0,0.000,0.000,ARI
30525,FF,90.1,2.40,5.74,"Erlin, Robbie",12.0,R,R,L,B,...,2500.0,6.0,54.54,39,4,4-Seam Fastball,162.0,0.000,0.060,ARI
30526,CU,75.9,2.52,5.49,"Erlin, Robbie",8.0,R,R,L,S,...,2688.0,6.0,54.53,39,3,Curveball,336.0,0.000,-0.078,ARI
30527,FT,90.2,2.60,5.59,"Erlin, Robbie",12.0,R,R,L,B,...,2241.0,6.2,54.28,39,2,2-Seam Fastball,136.0,0.000,0.030,ARI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103985,CU,77.0,3.34,5.55,"Dayton, Grant",13.0,R,R,L,B,...,2523.0,6.6,53.91,47,10,Curveball,337.0,0.000,0.060,WSH
103986,FF,92.2,3.30,5.57,"Dayton, Grant",12.0,R,R,L,S,...,2279.0,7.3,53.20,47,9,4-Seam Fastball,143.0,0.000,0.000,WSH
103987,FF,91.7,3.30,5.65,"Dayton, Grant",12.0,R,R,L,S,...,2281.0,6.5,54.01,47,8,4-Seam Fastball,148.0,0.000,0.000,WSH
103968,SL,82.0,2.21,6.32,"Smith, Will",13.0,R,L,L,B,...,2340.0,6.4,54.12,53,1,Slider,311.0,0.000,0.041,WSH
